In [63]:
import pandas as pd

df_crims = pd.read_csv('CMX_2024.csv')
df_crims = df_crims.dropna()
df_crims['fecha_hecho'] = pd.to_datetime(df_crims['fecha_hecho'])
df_crims['hora_hecho'] = df_crims['fecha_hecho'].dt.hour
df_crims['hora_hecho'] = pd.to_timedelta(df_crims['hora_hecho'])
df_crims['hora_hecho'] = df_crims['hora_hecho'].dt.total_seconds()



from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6)

# Fit the KMeans model to the 'time_seconds' variable
kmeans.fit(df_crims['hora_hecho'].values.reshape(-1, 1))

# Get the cluster labels
cluster_labels = kmeans.labels_

# Assign the cluster labels to the dataframe
df_crims['time_zones'] = cluster_labels

display(df_crims['time_zones'].value_counts())
# Map the cluster labels to time zones
time_zone_mapping = {0: 'mañana', 1: 'noche', 2: 'madrugada', 3: 'mediodia', 4: 'tarde', 5: 'medianoche'}
df_crims['time_zones'] = df_crims['time_zones'].map(time_zone_mapping)

display(df_crims['time_zones'].value_counts())

display(df_crims.head())

C:\Users\11ser\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1389: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


time_zones
0    127555
Name: count, dtype: int64

time_zones
mañana    127555
Name: count, dtype: int64

,anio_inicio,mes_inicio,fecha_inicio,hora_inicio,anio_hecho,mes_hecho,fecha_hecho,hora_hecho,delito,categoria_delito,...,agencia,unidad_investigacion,colonia_hecho,colonia_catalogo,alcaldia_hecho,alcaldia_catalogo,municipio_hecho,latitud,longitud,time_zones
0,2024,Enero,2024-01-01,00:20:00,2024.0,Enero,2024-01-01,0.0,PERDIDA DE LA VIDA POR OTRAS CAUSAS,HECHO NO DELICTIVO,...,UAT-TLP-3,UI-1SD,SAN MIGUEL TOPILEJO,Pueblo San Miguel Topilejo,TLALPAN,Tlalpan,CDMX,19.198611,-99.140064,mañana
1,2024,Enero,2024-01-01,01:12:00,2023.0,Diciembre,2023-12-28,0.0,HOMICIDIO CULPOSO POR TRÁNSITO VEHICULAR (COLI...,DELITO DE BAJO IMPACTO,...,UAT-AZ-4,UI-2SD,AERONÁUTICA MILITAR,Aeronautica Militar,VENUSTIANO CARRANZA,Venustiano Carranza,CDMX,19.420629,-99.115402,mañana
2,2024,Enero,2024-01-01,01:14:00,2023.0,Diciembre,2023-12-31,0.0,PERDIDA DE LA VIDA POR CAIDA,HECHO NO DELICTIVO,...,UAT-IZP-5,1 SIN DETENIDO,OJITO DE AGUA,Pueblo Santa Maria Aztahuacan,IZTAPALAPA,Iztapalapa,CDMX,19.347257,-99.024526,mañana
3,2024,Enero,2024-01-01,02:29:00,2023.0,Diciembre,2023-12-31,0.0,"ROBO EN EVENTOS MASIVOS (DEPORTIVOS, CULTURALE...",DELITO DE BAJO IMPACTO,...,UAT-CUH-6,UI-1SD,CENTRO,Centro,CUAUHTEMOC,Cuauhtémoc,CDMX,19.434052,-99.134837,mañana
4,2024,Enero,2024-01-01,03:26:00,2024.0,Enero,2024-01-01,0.0,PERDIDA DE LA VIDA POR PARO CARDIACO,HECHO NO DELICTIVO,...,UAT-MC-2,UI-1SD,LOMAS DE SAN BERNABÉ,Lomas De San Bernabe,LA MAGDALENA CONTRERAS,La Magdalena Contreras,CDMX,19.314159,-99.263989,mañana


In [54]:
from datetime import datetime
import nltk

# Convert 'fecha_hecho' to datetime


df_delict = df_crims[df_crims['categoria_delito'] != 'HECHO NO DELICTIVO']

df_bajo_impacto = df_delict[df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO']
small_delicts = df_bajo_impacto['delito'].value_counts()
filtered_delicts = small_delicts[small_delicts.index.str.contains('ROBO|ACOSO|HOMICICIO|PANDILLA', case=False)]

display(df_delict.shape)
# Filter out the delicts that are NOT in the filtered_delicts list
df_delict = df_delict[~((df_delict['categoria_delito'] == 'DELITO DE BAJO IMPACTO') & (~df_delict['delito'].isin(filtered_delicts.index)))]
display(df_delict.shape)

df_delict = df_delict.dropna(subset=['latitud', 'longitud'])
display(df_delict.shape)


#df_crimes = df_delict[['delito', 'longitud', 'latitud']]

display(df_delict['categoria_delito'].value_counts())

(124952, 22)

(41051, 22)

(41051, 22)

categoria_delito
DELITO DE BAJO IMPACTO                                     29084
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        4249
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        3421
VIOLACIÓN                                                   1170
ROBO A NEGOCIO CON VIOLENCIA                                1029
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        565
HOMICIDIO DOLOSO                                             475
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        372
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      201
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                195
ROBO A CASA HABITACIÓN CON VIOLENCIA                         121
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA       80
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 73
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      11
SECUESTRO                                                      5
Name: co

In [55]:
crime_category_to_weight = {
    'DELITO DE BAJO IMPACTO': 1,
    'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA': 3,
    'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA': 2,
    'ROBO A NEGOCIO CON VIOLENCIA': 3,
    'VIOLACIÓN': 4,
    'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA': 2,
    'HOMICIDIO DOLOSO': 5,
    'ROBO A REPARTIDOR CON Y SIN VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE MICROBÚS CON Y SIN VIOLENCIA': 2,
    'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO': 4,
    'ROBO A CASA HABITACIÓN CON VIOLENCIA': 3,
    'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA': 3,
    'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA': 3,
    'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA': 2,
    'SECUESTRO': 5
}

df_crimes_with_weight = df_delict.copy()
df_crimes_with_weight['weight'] = df_crimes_with_weight['categoria_delito'].map(crime_category_to_weight)
df_crimes_with_weight['weight'] = df_crimes_with_weight['weight'].fillna(1)

display(df_crimes_with_weight['weight'].value_counts())

weight
1.0    29285
3.0     5924
2.0     3997
4.0     1365
5.0      480
Name: count, dtype: int64

In [56]:
df_ponderado = df_crimes_with_weight[['delito', 'categoria_delito', 'longitud', 'latitud', 'weight', 'hora_hecho']]
df_ponderado.loc[df_ponderado['delito'].str.contains('CON', case=True), 'weight'] += 0.5

df_weights = df_ponderado[['latitud','longitud','weight','hora_hecho']]
display(df_weights['weight'].value_counts())

weight
1.0    26385
3.5     4943
2.0     3259
1.5     2900
4.0     1332
3.0      981
2.5      738
5.0      480
4.5       33
Name: count, dtype: int64

In [57]:
import geojson
from geojson import Feature, FeatureCollection, Point

def df_to_geojson(df):
    features = []
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            point = Point((row['longitud'], row['latitud']))
            features.append(
                Feature(geometry=point, properties={"weight": row['weight'], "hora_hecho": row['hora_hecho']})
            )
    return FeatureCollection(features)

geojson_data = df_to_geojson(df_weights)

# Save to a GeoJSON file
with open('crimes.geojson', 'w') as f:
    geojson.dump(geojson_data, f)

In [20]:
def filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max):
    filtered_features = []
    for feature in geojson_data['features']:
        lon, lat = feature['geometry']['coordinates']
        if lat_min <= lat <= lat_max and lon_min <= lon <= lon_max:
            filtered_features.append(feature)
    
    return geojson.FeatureCollection(filtered_features)

# Example usage
lat_min, lat_max = 19.0, 20.0
lon_min, lon_max = -100.0, -98.0
filtered_geojson_data = filter_geojson_by_coordinates(geojson_data, lat_min, lat_max, lon_min, lon_max)

# Save the filtered GeoJSON to a new file
with open('filtered_crimes.geojson', 'w') as f:
    geojson.dump(filtered_geojson_data, f)

In [21]:
import folium

# Crear un mapa centrado en el área filtrada
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Añadir el archivo GeoJSON al mapa
folium.GeoJson('filtered_crimes.geojson').add_to(m)

# Mostrar el mapa


In [22]:
import folium

def visualize_map(df):
    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=12)
    
    # Add points to the map
    for _, row in df.iterrows():
        if not pd.isna(row['latitud']) and not pd.isna(row['longitud']):
            folium.Marker(
                location=[row['latitud'], row['longitud']],
                popup=row['delito'],
                tooltip=row['categoria_delito']
            ).add_to(m)
    
    return m

# Example usage
map_visualization = visualize_map(df_crims)
map_visualization